<a href="https://colab.research.google.com/github/martine-augustin/Marionnaud/blob/main/copie_de_marionnaud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Marionnaud projet


## Variable et installation

In [ ]:
# Install
!pip install beautifulsoup4
!pip install lxml




In [ ]:
# Import
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
import random
import csv
import pandas as pd

In [ ]:
#Constante


## Fonction

In [ ]:
def unAvis(section):
    global id_avis

    img_star = section.find('img', class_='CDS_StarRating_starRating__614d2e')
    if not img_star:
        return None

    nb_etoile_text = img_star.get('alt')
    nb_etoile = nb_etoile_text[5]

    date_commentaire_tag = section.find('span')
    if not date_commentaire_tag or not nb_etoile.isdigit():
        return None

    date_commentaire = date_commentaire_tag.get_text(strip=True)
    if not date_commentaire[0].isdigit():
        return None

    id_avis += 1
    titre_tag = section.find('h2')
    titre = titre_tag.get_text(strip=True) if titre_tag else None

    commentaire_tag = section.find('p')
    commentaire_text = commentaire_tag.get_text(strip=True) if commentaire_tag else None

    return [id_avis, page_avis, nb_etoile, date_commentaire, titre, commentaire_text]

In [ ]:
def PageAvis(url):
    liste_des_avis_page = []

    # Récupérer le contenu de la page avec requests
    response = requests.get(url)

    if response.status_code != 200:
        return "Error"

    # Parser le HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    sections = soup.find_all('section')  # toutes les sections

    for section in sections:
        avis = unAvis(section)
        if avis:  # si unAvis renvoie bien une liste
            liste_des_avis_page.append(avis)

    return liste_des_avis_page

In [ ]:
def CreateCSV(name_file, tab_avis):
  with open(name_file, mode="w", newline="", encoding="utf-8") as fichier_csv:
    writer = csv.writer(fichier_csv)


    # Écrire une ligne d'en-tête
    writer.writerow(["Avis ID", "Page ID", "Nombre etoile", "Date", "Titre", "Commentaire"])

    # Écrire les données
    for ligne in tab_avis:
        writer.writerow(ligne)

In [ ]:
def OpenCSV(filename):
    """
    Ouvre un fichier CSV et retourne un DataFrame pandas
    """
    try:
        df = pd.read_csv(filename, encoding="utf-8")
        return df
    except FileNotFoundError:
        print(f"Erreur : le fichier {filename} n'existe pas.")
        return None
    except pd.errors.EmptyDataError:
        print(f"Erreur : le fichier {filename} est vide.")
        return None

## Scrapping

In [ ]:
def methode_de_scrapping_1 (url) :
  # Variable
  id_avis = 0
  page_avis_end = False
  page_avis = 0
  page_avis_max = 10
  liste_des_avis = list()

  #On recupere les avis de toutes les pages Marionnaud
  while page_avis_end == False:
    page_avis += 1
    url_final = url + str(page_avis)

    result = PageAvis(url)
    if(result != "Error"):
    #On fusionne notre tableau temporaire avec le tableau principal

      if liste_des_avis:
        liste_des_avis = liste_des_avis + result
      else:
        liste_des_avis = result

      # Génère un nombre entier aléatoire entre 45 et 60
      delai = random.randint(45, 60)
      print("Début du timeur de la page : " + str(page_avis) + " en cours ...")
      time.sleep(60)  # attend 5 secondes

    else:
      page_avis_end = True


    if(page_avis_max < page_avis):
      page_avis_end = True

  print("--------------------------")
  print("url :" + url)
  print(result)
  print("nombre élément liste_des_avis : " + str(len(liste_des_avis)))

  return liste_des_avis


In [ ]:
def Scrapping() :
  # Marionnaud
  liste_des_avis_marionnaud = methode_de_scrapping_1('https://fr.trustpilot.com/review/www.marionnaud.com?page=')
  CreateCSV('marionnaud.csv',liste_des_avis_marionnaud)

  # Sephora
  liste_des_avis_sephora = methode_de_scrapping_1('https://fr.trustpilot.com/review/www.sephora.fr?page=')
  CreateCSV('sephora.csv', liste_des_avis_sephora)


In [ ]:
Scrapping()


Début du timeur de la page : 1 en cours ...
Début du timeur de la page : 2 en cours ...
Début du timeur de la page : 3 en cours ...
Début du timeur de la page : 4 en cours ...
Début du timeur de la page : 5 en cours ...
Début du timeur de la page : 6 en cours ...
Début du timeur de la page : 7 en cours ...
Début du timeur de la page : 8 en cours ...
Début du timeur de la page : 9 en cours ...
Début du timeur de la page : 10 en cours ...
Début du timeur de la page : 11 en cours ...
--------------------------
url :https://fr.trustpilot.com/review/www.marionnaud.com?page=
[[341, 1, '1', '24 février 2026', 'Bonjour,', 'Bonjour,Je souhaite partager mon expérience suite au retrait de ma commande hier chez Marionnaud, 13 Place François Mitterrand, Vienne 38200.J’ai été accueillie par deux conseillères : une jeune femme très gentille, souriante et professionnelle, que je remercie sincèrement pour son accueil. En revanche, j’ai été déçue par l’attitude d’une autre conseillère, un peu plus âgée,

In [ ]:
url = 'https://fr.trustpilot.com/review/www.marionnaud.com?page='

print(liste_des_avis)
CreateCSV(liste_des_avis)

In [ ]:
# Ouvrir le CSV créé précédemment
df_avis = OpenCSV("avis.csv")

# Vérifier que ça a fonctionné
if df_avis is not None:
    print(df_avis.head())  # affiche les 5 premières lignes


Erreur : le fichier avis.csv n'existe pas.
